In [57]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [58]:
name_list=['Frequency',' Angle of attack','Chord length','Free-stream velocity','Suction side displacement thickness','output']

In [59]:
train=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat",names=name_list,sep='\t')

In [60]:
train.isnull().sum()

Frequency                              0
 Angle of attack                       0
Chord length                           0
Free-stream velocity                   0
Suction side displacement thickness    0
output                                 0
dtype: int64

In [55]:
train.head(5)

,Frequency,Angle of attack,Chord length,Free-stream velocity,Suction side displacement thickness,output
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [56]:
features=train.drop("output",axis=1)
outcomes=train["output"].values
features.head(5)

,Frequency,Angle of attack,Chord length,Free-stream velocity,Suction side displacement thickness
0,800,0.0,0.3048,71.3,0.002663
1,1000,0.0,0.3048,71.3,0.002663
2,1250,0.0,0.3048,71.3,0.002663
3,1600,0.0,0.3048,71.3,0.002663
4,2000,0.0,0.3048,71.3,0.002663


In [36]:
for i in features.columns: 
    scaler = preprocessing.StandardScaler()
    features[i] = scaler.fit_transform(features[i])

/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/bruce/anaconda

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, outcomes, test_size=0.25, random_state=10)

# Random forest by xgboost

In [30]:
from xgboost import XGBRegressor

final_model = XGBRegressor(n_estimators=500,num_boost_round=1,max_depth=5,subsample=0.632,colsample_bytree=0.8)
final_model.fit(X_train.values, Y_train.astype(int))
y_pred = final_model.predict(X_test.values)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

print(np.sqrt(mean_squared_error(Y_test, y_pred)))
print(explained_variance_score(Y_test, y_pred))



1.52236529279
0.948011264516


# Decision tree by Xgboost

In [21]:
from xgboost import XGBRegressor

final_model = XGBRegressor(n_estimators=1,num_boost_round=1,max_depth=5,subsample=1,colsample_bytree=1)
final_model.fit(X_train.values, Y_train.astype(int))
y_pred = final_model.predict(X_test.values)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

print(np.sqrt(mean_squared_error(Y_test, y_pred)))
print(explained_variance_score(Y_test, y_pred))




112.257415722
0.0


# Bagged tree by Xgboost

In [22]:
from xgboost import XGBRegressor

final_model = XGBRegressor(n_estimators=500,num_boost_round=1,max_depth=5,subsample=0.632,colsample_bytree=1)
final_model.fit(X_train.values, Y_train.astype(int))
y_pred = final_model.predict(X_test.values)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

print(np.sqrt(mean_squared_error(Y_test, y_pred)))
print(explained_variance_score(Y_test, y_pred))




1.42981530771
0.954888770835


# Standard feed-forward neural network

In [40]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasRegressor


def base_model():
    model = Sequential()
    model.add(Dense(128, input_dim=5, init='normal', activation='sigmoid'))
    model.add(Dense(128, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer = 'adam')
    return model

estimator = KerasRegressor(build_fn = base_model,epochs=200,batch_size=128,)
estimator.fit(X_train.values, Y_train )
y_pred = estimator.predict(X_test.values )
rmse = mean_squared_error( Y_test, y_pred)



/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="sigmoid", kernel_initializer="normal", input_dim=5)`
/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="normal")`
/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


Epoch 1/200
1127/1127 [==============================] - 0s - loss: 15488.4316     
Epoch 2/200
1127/1127 [==============================] - 0s - loss: 15165.7769     
Epoch 3/200
1127/1127 [==============================] - 0s - loss: 14771.9302     
Epoch 4/200
1127/1127 [==============================] - 0s - loss: 14251.9114     
Epoch 5/200
1127/1127 [==============================] - 0s - loss: 13590.3742     
Epoch 6/200
1127/1127 [==============================] - 0s - loss: 12782.6811     
Epoch 7/200
1127/1127 [==============================] - 0s - loss: 11

In [41]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

print(np.sqrt(mean_squared_error(Y_test, y_pred)))
print(explained_variance_score(Y_test, y_pred))

4.66728045589
0.464944022161


# New model

In [42]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint,History
from keras.layers import Dense, Activation, Dropout, Input
from keras import optimizers
import keras
from keras.models import Model

In [44]:
def new_model():
    input_1 = Input(shape=(1,))
    input_2 = Input(shape=(1,))
    input_3 = Input(shape=(1,))
    input_4 = Input(shape=(1,))
    input_5 = Input(shape=(1,))
    
    hidden_1 = Dense(32, activation='sigmoid')(input_1)
    hidden_2 = Dense(32, activation='sigmoid')(input_2)
    hidden_3 = Dense(32, activation='sigmoid')(input_3)
    hidden_4 = Dense(32, activation='sigmoid')(input_4)
    hidden_5 = Dense(32, activation='sigmoid')(input_5)
    
    x = keras.layers.concatenate([hidden_1,hidden_2,hidden_3,hidden_4,hidden_5])
    x = Dense(128, activation='relu')(x)
    x = Dense(1)(x)
    model = Model(inputs=[input_1,input_2,input_3,input_4,input_5], outputs=[x])
    model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(lr=0.001))
    return model

value_list=[X_train[['Frequency']].values,
            X_train[[' Angle of attack']].values,
            X_train[['Chord length']].values,
            X_train[['Free-stream velocity']].values,
            X_train[['Suction side displacement thickness']].values]
    
value_list_test=[X_test[['Frequency']].values,
            X_test[[' Angle of attack']].values,
            X_test[['Chord length']].values,
            X_test[['Free-stream velocity']].values,
            X_test[['Suction side displacement thickness']].values,
           ]

In [49]:
estimator = KerasRegressor(build_fn = new_model,epochs=200,batch_size=32,)
estimator.fit(value_list, Y_train )
y_pred = estimator.predict(value_list_test)



Epoch 1/200
1127/1127 [==============================] - 0s - loss: 13208.8897     
Epoch 2/200
1127/1127 [==============================] - 0s - loss: 7477.2454      
Epoch 3/200
1127/1127 [==============================] - 0s - loss: 2178.6741     
Epoch 4/200
1127/1127 [==============================] - 0s - loss: 193.2489     
Epoch 5/200
1127/1127 [==============================] - 0s - loss: 39.4438     

In [50]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

print(np.sqrt(mean_squared_error(Y_test, y_pred)))
print(explained_variance_score(Y_test, y_pred))

4.49900771002
0.503277730067


# Neural network ensemble by bagging 

In [73]:
name_list=['Frequency',' Angle of attack','Chord length','Free-stream velocity','Suction side displacement thickness','output']

In [74]:
train=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat",names=name_list,sep='\t')

In [75]:
scaler = preprocessing.StandardScaler()
train['Frequency'] = scaler.fit_transform(train['Frequency'])
train[' Angle of attack'] = scaler.fit_transform(train[' Angle of attack'])
train['Chord length'] = scaler.fit_transform(train['Chord length'])
train['Suction side displacement thickness'] = scaler.fit_transform(train['Suction side displacement thickness'])
train['Free-stream velocity'] = scaler.fit_transform(train['Free-stream velocity'])

/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/bruce/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your 

In [76]:
train.head(5)

,Frequency,Angle of attack,Chord length,Free-stream velocity,Suction side displacement thickness,output
0,-0.662023,-1.146403,1.799299,1.312935,-0.644805,126.201
1,-0.598561,-1.146403,1.799299,1.312935,-0.644805,125.201
2,-0.519235,-1.146403,1.799299,1.312935,-0.644805,125.951
3,-0.408177,-1.146403,1.799299,1.312935,-0.644805,127.591
4,-0.281255,-1.146403,1.799299,1.312935,-0.644805,127.461


In [77]:
from sklearn.model_selection import train_test_split
data_train,data_val=train_test_split(train,test_size=0.25, random_state=10)
X_val=data_val.drop(['output'], axis=1).values
y_val=data_val['output'].ravel()

In [78]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error



def train_nn_simple(data_train,X_val,y_val):
    data_train_new=data_train.sample(frac=0.632,replace=True)
    X_train=data_train_new.drop(['output'], axis=1).values
    y_train=data_train_new['output'].ravel()
    estimator = KerasRegressor(build_fn = base_model,epochs=200,batch_size=32,)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_val)
    rmse =np.sqrt(mean_squared_error( y_val, y_pred)) 
    return rmse
    
    
def base_model():
    model = Sequential()
    model.add(Dense(128, input_dim=5, init='normal', activation='sigmoid'))
    model.add(Dense(128, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer = 'adam')
    return model
  

In [79]:
result=[]
for i in range(50):
    rmse=train_nn_simple(train,X_val,y_val)
    result.append(rmse)

/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="sigmoid", kernel_initializer="normal", input_dim=5)`
/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="normal")`
/Users/bruce/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


Epoch 1/200
950/950 [==============================] - 0s - loss: 15080.6766     
Epoch 2/200
950/950 [==============================] - 0s - loss: 13021.1197     
Epoch 3/200
950/950 [==============================] - 0s - loss: 9322.1599      
Epoch 4/200
950/950 [==============================] - 0s - loss: 4929.5594     
Epoch 5/200
950/950 [==============================] - 0s - loss: 1607.8956     
Epoch 6/200
950/950 [==============================] - 0s - loss: 258.3700     
Epoch 7/200
950/950 [==============

In [80]:
rmse=np.sum(result)/50
rmse

4.5698448581115674